In [ ]:
import pandas as pd
import numpy as np

In [2]:
datapath = 'tsinghua-1error.csv'
df = pd.read_csv(datapath)

In [3]:
df.columns = ['text','category']
df["TextLength"]= df['text'].str.len() #加入TextLength特征，用来判断训练向量的长度大概设多大就够用。
df.head()

,text,category,TextLength
0,高速一副大队长曾昭智接受采访说，现时不是春运，所以相对来说超载的现象比较少，比较突出的问题是...,通顺,52.0
1,高速一副大队长曾昭智接受采访说，现时不是春运，所以相对来说超载行车道的现象比较少，比较突出的...,不通顺,55.0
2,另有一个特点就是，实验题目的命题坚持做到考生动手做过实验才能得高分,通顺,33.0
3,另有一个特点就是，实验题目的一类命题坚持做到考生动手做过实验才能得高分,不通顺,35.0
4,记者从规划委房山分局证实，加州水郡四期规划方案中，有规划城市支路一条，东至广阳北大街(规划路...,通顺,68.0


In [4]:
df['text'] = df['text'].astype('str')
df['TextLength'].describe()

count    1.299325e+06
mean     4.818062e+01
std      2.986141e+01
min      6.000000e+00
25%      2.800000e+01
50%      4.200000e+01
75%      6.100000e+01
max      8.390000e+02
Name: TextLength, dtype: float64

可见句子平均长度在48个字，75%分位数在61个字，故句子向量长度我设置了80个字，更省时间

In [5]:
df['category'].value_counts() / df.shape[0]

通顺     0.5
不通顺    0.5
Name: category, dtype: float64

In [6]:
df.shape

(1299442, 3)

In [7]:
a=list(df['category'])
b=set(a)
c=list(b)
labels ={}
for i in range(len(c)):
    labels[c[i]] = i
labels

{'不通顺': 0, '通顺': 1}

In [8]:
np.random.seed(8)
fraction = 1
df_train, df_val, df_test = np.split(df.sample(frac=fraction , random_state=8), [int(0.8*fraction*len(df)), int(0.9*fraction*len(df))])

In [9]:
len(df_test)

129945

In [10]:
len(df_train)

1039553

In [11]:
xxx = list(df['category'])
aaa=set(xxx)
bbb=list(aaa)
labels ={}
for i in range(len(bbb)):
    labels[bbb[i]] = i
labels

{'不通顺': 0, '通顺': 1}

In [12]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 80, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

/root/miniconda3/envs/my-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, len(labels))
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [14]:
from torch.optim import 
from tqdm import tqdm
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True, num_workers=1, pin_memory=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
            
            model = model.cuda()
            criterion = criterion.cuda()
            #     #新加的半精度训练器
            # scaler = torch.cuda.amp.GradScaler()
            # autocast = torch.cuda.amp.autocast

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                
                optimizer.zero_grad()
                
                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                
                batch_loss.backward()
                optimizer.step()
                total_loss_train += batch_loss.item()

#                 optimizer.zero_grad()
                
#                 with autocast():
#                     output = model(input_id, mask)
#                     batch_loss = criterion(output, train_label.long())
                
#                 scaler.scale(batch_loss).backward()
                
#                 scaler.step(optimizer)
                
#                 scaler.update()
                
#                 total_loss_train += batch_loss.item()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
            
            torch.save(model.state_dict(),f'./thu_1error_relu_epoches/epoch{epoch_num+1}.pth')
            print(f'epoch{epoch_num+1}保存成功')

In [15]:
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 64973/64973 [2:12:21<00:00,  8.18it/s]  


Epochs: 1 | Train Loss:  0.037                 | Val Loss:  0.035                 | Val Accuracy:  0.651
epoch1保存成功


100%|██████████| 64973/64973 [2:12:43<00:00,  8.16it/s]  


Epochs: 2 | Train Loss:  0.033                 | Val Loss:  0.034                 | Val Accuracy:  0.653
epoch2保存成功


100%|██████████| 64973/64973 [2:10:09<00:00,  8.32it/s]  


Epochs: 3 | Train Loss:  0.032                 | Val Loss:  0.034                 | Val Accuracy:  0.654
epoch3保存成功


100%|██████████| 64973/64973 [2:10:02<00:00,  8.33it/s]  


Epochs: 4 | Train Loss:  0.031                 | Val Loss:  0.035                 | Val Accuracy:  0.653
epoch4保存成功


100%|██████████| 64973/64973 [2:10:09<00:00,  8.32it/s]  


Epochs: 5 | Train Loss:  0.030                 | Val Loss:  0.036                 | Val Accuracy:  0.652
epoch5保存成功


In [16]:
def evaluate(model, test_data):
    
    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=10)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

Test Accuracy:  0.650
